In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import dice_ml
from dice_ml.utils import helpers # helper functions
from dice_ml import Data,Model,Dice
import numpy as np
from xgboost import XGBClassifier
import time
import threading
from joblib import Parallel, delayed

import os
import json
import openai
from openai import AzureOpenAI
#from langchain.llms import AzureOpenAI
#from langchain_openai import AzureChatOpenAI
import pandas as pd 




: 

In [ ]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
dataframe_heart_disease = pd.read_csv("heart_statlog_cleveland_hungary_final.csv")

In [ ]:


dataframe_heart_disease =  dataframe_heart_disease.dropna()
dataframe_heart_disease =  dataframe_heart_disease.drop_duplicates()
dataframe_heart_disease = dataframe_heart_disease[dataframe_heart_disease['chol'] !=0]
dataframe_heart_disease = dataframe_heart_disease[dataframe_heart_disease['trestbps'] !=0]

In [ ]:
# Split the data into features and target label
y = dataframe_heart_disease.target
X = dataframe_heart_disease.drop(['target'], axis=1)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
numerical = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

categorical = X_train.columns.difference(numerical)

In [ ]:
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])


In [ ]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', XGBClassifier())])
xgb_model = clf.fit(X_train, y_train)


In [ ]:
param_grid = {
    'classifier__max_depth': [3, 4, 5],
    'classifier__learning_rate': [0.01, 0.1, 0.5],
    'classifier__n_estimators': [100, 200, 300],
    'classifier__gamma': [0, 0.1, 0.5]
}


In [ ]:
# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(xgb_model, param_grid=param_grid)
grid_search.fit(X_train, y_train)


# Print the best parameters and the score on the test set
print("Best parameters: ", grid_search.best_params_)
print("Test set score: ", grid_search.score(X_test, y_test))

In [ ]:
param_grid = {
    'classifier__max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'classifier__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.5],
    'classifier__n_estimators': [50, 100, 200, 300, 400, 500],
    'classifier__gamma': [0, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5],
    'classifier__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'classifier__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'classifier__min_child_weight': [1, 2, 3, 4, 5],
    'classifier__reg_alpha': [0, 0.01, 0.1, 1, 10],
    'classifier__reg_lambda': [0, 0.01, 0.1, 1, 10]
}

In [ ]:
# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(xgb_model, param_grid=param_grid)
grid_search.fit(X_train, y_train)


# Print the best parameters and the score on the test set
print("Best parameters: ", grid_search.best_params_)
print("Test set score: ", grid_search.score(X_test, y_test))

In [ ]:
# Import necessary libraries
from skopt import BayesSearchCV
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

# Define the parameter space for Bayesian search
param_space = {
    'classifier__max_depth': (3, 10),
    'classifier__learning_rate': (0.001, 0.5, 'log-uniform'),
    'classifier__n_estimators': (50, 500),
    'classifier__gamma': (0, 0.5, 'uniform'),
    'classifier__subsample': (0.6, 1.0, 'uniform'),
    'classifier__colsample_bytree': (0.6, 1.0, 'uniform'),
    'classifier__min_child_weight': (1, 5),
    'classifier__reg_alpha': (0, 10, 'log-uniform'),
    'classifier__reg_lambda': (0, 10, 'log-uniform')
}

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', XGBClassifier())])

# Perform Bayesian search
bayes_search = BayesSearchCV(
    estimator=clf,
    search_spaces=param_space,
    n_iter=50,  # Number of iterations
    cv=5,  # Cross-validation splitting strategy
    n_jobs=-1,  # Use all available cores
    verbose=0,
    random_state=42
)

# Fit the model using Bayesian search
bayes_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", bayes_search.best_params_)

# Use the best estimator for predictions
best_model = bayes_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_test, y_pred))

In [ ]:
# create the pipeline with the XGBClassifier
pipeline = Pipeline(steps=[
    ('preprocessor', transformations),
    ('classifier', XGBClassifier(max_depth=5, learning_rate=0.5, n_estimators=200, gamma=0))
])


In [ ]:
# train the model on the entire training set
xgb_pipeline = pipeline.fit(X_train, y_train)

# make predictions on the test set
y_pred = xgb_pipeline.predict(X_test)


In [ ]:
X_high_risk_tp = X_test[(y_pred == 1) & (y_test == 1)].reset_index().drop(['index'], axis=1)
#X_high_risk_tp = X_test[(y_pred == 1) & (y_test == 1)]

In [ ]:
train_data = pd.concat([X_train, y_train], axis=1)


# Create a DICE data object
d = Data(dataframe=pd.DataFrame(train_data, columns=dataframe_heart_disease.columns), continuous_features=['age', 'trestbps', 'chol', 'thalach', 'oldpeak'],outcome_name='target')

# Create a DICE model object
m = Model(model=xgb_pipeline, backend="sklearn")

Ideal Countefactuals

In [ ]:
import threading
import queue

df_no_counterfactuals = pd.DataFrame(columns=X_high_risk_tp.columns)
def generate_cf(test_instance, timeout=10):
    q = queue.Queue()

    def target():
        try:
            result = Dice(d, m, method='genetic').generate_counterfactuals(test_instance, total_CFs=3, desired_class="opposite",
                                                                           features_to_vary=["trestbps", "chol", "thalach"],
                                                                           #diversity_weight=5, proximity_weight=2, sparsity_weight=5,
                                                                           permitted_range={"trestbps": [80, 120], "chol": [150, 200], "thalach": [120, 220 - test_instance['age'].values[0]]}
                                                                           )
            q.put(result)
        except Exception as e:
            print("No counterfactuals found for test instance:", test_instance)
            df_no_counterfactuals.append(test_instance)
            q.put(None)

    # Start a new thread to run the target function
    thread = threading.Thread(target=target)
    thread.start()

    # Wait for the thread to finish or raise a timeout exception
    thread.join(timeout)

    if thread.is_alive():
        # The thread is still running, so raise a timeout exception
        print("No counterfactuals found for test instance-timed out:", test_instance)
        df_no_counterfactuals.append(test_instance)
        q.put(None)
    else:
        # The thread has finished, so return the result
        return q.get()

In [ ]:

exps_ideal = []
num_cores = -1

# Iterate over each instance of X_high_risk and generate counterfactuals
for i in range(len(X_high_risk_tp)):
    test_instance = X_high_risk_tp.iloc[[i]]
    print(i)
    exp = Parallel(n_jobs=num_cores)(delayed(generate_cf)(test_instance) for i in range(1))
    exps_ideal.append(exp)

In [ ]:
num_none = sum(exp[0] is None for exp in exps_ideal)
print("Number of None values in exps_ideal:", num_none)

In [ ]:
print((96-18)/96)

In [ ]:
validity_list = []
for exps in exps_ideal:
    
    if exps[0] is not None:
       exp_df = exps[0].cf_examples_list[0].final_cfs_df
       validity_list.append(xgb_pipeline.predict(exp_df)[0])

In [ ]:
mean_values_list = []
median_values_list = []
std_values_list = []

for exps in exps_ideal:
    if exps[0] is not None:
        exp_df = exps[0].cf_examples_list[0].final_cfs_df

        # Calculate the mean
        mean_values = exp_df[['chol', 'trestbps', 'thalach']].mean()
        mean_values_list.append(mean_values)

        # Calculate the median
        median_values = exp_df[['chol', 'trestbps', 'thalach']].median()
        median_values_list.append(median_values)

        # Calculate the standard deviation
        std_values = exp_df[['chol', 'trestbps', 'thalach']].std()
        std_values_list.append(std_values)

# Convert lists to DataFrames
mean_values_df = pd.DataFrame(mean_values_list)
median_values_df = pd.DataFrame(median_values_list)
std_values_df = pd.DataFrame(std_values_list)

# Calculate and print the mean of means, median of medians, and std dev of std devs
print('Mean of means:\n', mean_values_df.mean())
print('\nMedian of medians:\n', median_values_df.median())
print('\nStandard deviation of standard deviations:\n', std_values_df.std())

In [ ]:
import pandas as pd
import numpy as np

# Initialize an empty DataFrame
all_data_df = pd.DataFrame()

statistics = {}

for exps in exps_ideal:
    if exps[0] is not None:
        exp_df = exps[0].cf_examples_list[0].final_cfs_df
        all_data_df = pd.concat([all_data_df, exp_df[['chol', 'trestbps', 'thalach']]])


# Calculate and store the mean, median, and std dev
for column in ['chol', 'trestbps', 'thalach']:
    statistics[column] = {
        'mean': np.mean(all_data_df[column]),
        'median': np.median(all_data_df[column]),
        'std_dev': np.std(all_data_df[column])
    }

print(statistics)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


# Convert the dictionary to a DataFrame
statistics_df = pd.DataFrame(statistics)

# Create a new figure
plt.figure(figsize=(15, 5))

# Plot the mean values
plt.subplot(1, 3, 1)
statistics_df.loc['mean'].plot(kind='bar')
plt.title('Mean')
plt.ylabel('Value')

# Plot the median values
plt.subplot(1, 3, 2)
statistics_df.loc['median'].plot(kind='bar')
plt.title('Median')

# Plot the standard deviation values
plt.subplot(1, 3, 3)
statistics_df.loc['std_dev'].plot(kind='bar')
plt.title('Standard Deviation')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
exps_ideal[0][0].visualize_as_dataframe()

In [ ]:
percentage = sum(validity_list) / len(validity_list) * 100
print("Number of Valid Ideal Counterfactuals: {:.2f}%".format(100-percentage))

LLM Counterfactuals

In [ ]:
df_tp = X_high_risk_tp.copy()  # Create a copy of the DataFrame to avoid modifying the original one
df_tp['target'] = 1  # Append a new column 'target' with all values as 1


In [ ]:
import json
def read_data():
    dict_strings = []
    #df_tp = pd.read_csv("heart_statlog_cleveland_hungary_final.csv")
    # Update 'sex' column
    df_tp['sex'] = df_tp['sex'].replace({1: 'Male', 0: 'Female'})
    # Update 'cp' column
    df_tp['cp'] = df_tp['cp'].replace({1: 'typical angina', 2: 'atypical angina', 3: 'non-anginal pain', 4: 'asymptomatic'})
    # Update 'fbs' column
    df_tp['fbs'] = df_tp['fbs'].replace({1: 'True', 0: 'False'})
    # Update 'restecg' column
    df_tp['restecg'] = df_tp['restecg'].replace({0: 'normal', 1: 'ST-T wave abnormality', 2: 'probable or definite left ventricular hypertrophy'})
    # Update 'exang' column
    df_tp['exang'] = df_tp['exang'].replace({1: 'yes', 0: 'no'})
    # Update 'slope' column
    df_tp['slope'] = df_tp['slope'].replace({1: 'upsloping', 2: 'flat', 3: 'downsloping'})
    # Update 'target' column
    df_tp['target'] = df_tp['target'].replace({1: 'yes', 0: 'no'})
    df_tp_risk = df_tp[df_tp['target'] == 'yes'].reset_index(drop=True)
    for i in range(len(df_tp_risk)):
        patient_records = df_tp_risk.iloc[i]
        dict_string = json.dumps(patient_records.to_dict())
        dict_strings.append(dict_string)
    return dict_strings


In [ ]:
patient_data = read_data()

In [ ]:
import openai
import json
import os
import concurrent.futures
import time


    
def create_patient_description (patient_record):
    # Convert the patient record to a dictionary string
    
    #print(i)
    dict_string = json.dumps(patient_record)

    # Create the message text
    message_text = [{
        "role": "system",
        "content": f"""You are a helpful assistant. 
        Describe this patient info
        ---
        {dict_string} 
        ---
        in plain english for me. Be precise and concise.
        """}]

    # Create the chat completion
    completion = client.chat.completions.create(
        model="gpt-4-32k-deployment",
        messages=message_text,
        temperature=0.0,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    # Return the generated description
    return completion.choices[0].message.content




def create_patient_description_with_retry(patient_record):
    retries = 0
    while True:
        try:
            return create_patient_description(patient_record)
        except Exception as e:
            if '429' in str(e):
                print(f"Rate limit exceeded. Retrying after {2**retries} seconds.")
                time.sleep(2**retries)
                retries += 1
            else:
                print(f"An error occurred: {e}")
                return None

def create_patient_descriptions(patient_records):
    # Create a thread pool executor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        patient_descriptions = list(executor.map(create_patient_description_with_retry, patient_records))

    return patient_descriptions

In [ ]:
patient_descriptions = []
for i in range(len(patient_data)):
    patient_record = json.loads(patient_data[i])
    patient_description = create_patient_description(patient_record)
    patient_descriptions.append(patient_description)

In [ ]:
def create_patient_cf_ideal (patient_record):
    # Convert the patient record to a dictionary string 

    message_text = [{"role":"system",
        "content":f"""You are a helpful assistant in Counterfactual Reasoning. 
        Here are the patient details:
        --- 
        {patient_record}
        ---
        What should the patient do specifically to prevent heart disease?
        Provide only three counterfactuals in precise comma separated manner in the following json format:
        ---
        {{"Preventive Measures": [[{{"Cholestrol":"chol1"}},{{"Resting Blood Pressure":"trestbps1"}},{{"Max Heart Rate":"thalach1"}}],
        [{{"Cholestrol":"chol2"}},{{"Resting Blood Pressure":"trestbps2"}},{{"Max Heart Rate":"thalach2"}}],
        [{{"Cholestrol":"chol3"}},{{"Resting Blood Pressure":"trestbps3"}},{{"Max Heart Rate":"thalach3"}}]]}}
        ---
        
        Generate three counterfactuals where the value for 'Cholestrol' is less than the value for 'chol' and also less than 200, the value for 'Resting Blood Pressure' is less than the value for 'trestbps' and also less than 120, and the value for 'Max Heart Rate' is greater than the value for 'thalach' and less than the difference between 220 and the patient's age.For example, if a patient's 'thalach' is 150 and their age is 50, the 'Max Heart Rate' in the counterfactual should be a number greater than 150 but less than 170 (which is 220 minus 50).
        Satisfy all the constraints and provide the counterfactuals.
        Be consitent in formatting. Don't provide general advice and say don't know if you can't provide a specific number.
        """}]

    completion = client.chat.completions.create(
            model="gpt-4-32k-deployment",
            messages=message_text,
            temperature=0.0,
            max_tokens=800,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )
    
    return completion.choices[0].message.content

In [ ]:
patient_cvd_counterfactual_ideal = []
for idx,patient_record in enumerate(patient_descriptions):
    print(idx)
    patient_cvd_counterfactual_ideal.append(create_patient_cf_ideal(patient_record))

In [ ]:

def get_validity(patient_cvd_counterfactual_ideal,xgb_pipeline):
    
    validity_list = []

    for i in range(len(patient_cvd_counterfactual_ideal)):

        data = json.loads(patient_cvd_counterfactual_ideal[i])['Preventive Measures']
        # Flatten each sub-list into a single dictionary
        flattened_data = [dict(item for d in sublist for item in d.items()) for sublist in data]

        # Convert the list of dictionaries into a DataFrame
        df = pd.DataFrame(flattened_data)

        # Get the first instance of X_high_risk_tp
        first_instance = X_high_risk_tp.iloc[i]
        counterfactuals_df = pd.DataFrame()
        
        

        # Create three counterfactuals
        for i in range(len(df)):
            # Copy the first instance
            counterfactual = first_instance.copy()
            
            # Update the values
            counterfactual['trestbps'] = df.iloc[i]['Resting Blood Pressure']
            counterfactual['chol'] = df.iloc[i]['Cholestrol']
            counterfactual['thalach'] = df.iloc[i]['Max Heart Rate']
            
            # Append the counterfactual to X_high_risk_tp
            #X_high_risk_tp = X_high_risk_tp.append(counterfactual, ignore_index=True)
            counterfactuals_df = counterfactuals_df.append(counterfactual, ignore_index=True)

        validity_list.append(xgb_pipeline.predict(counterfactuals_df)[0])
        
        
    
    print(len(validity_list))
    percentage = sum(validity_list) / len(validity_list) * 100

    return percentage,validity_list
    



In [ ]:
data = json.loads(patient_cvd_counterfactual_ideal[3])['Preventive Measures']
# Flatten each sub-list into a single dictionary
flattened_data = [dict(item for d in sublist for item in d.items()) for sublist in data]

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(flattened_data)

In [ ]:
counterfactuals_df = pd.DataFrame()
        
        
first_instance = X_high_risk_tp.iloc[3]
# Create three counterfactuals
for i in range(len(df)):
    # Copy the first instance
    counterfactual = first_instance.copy()
    
    # Update the values
    counterfactual['trestbps'] = df.iloc[i]['Resting Blood Pressure']
    counterfactual['chol'] = df.iloc[i]['Cholestrol']
    counterfactual['thalach'] = df.iloc[i]['Max Heart Rate']
    
    # Append the counterfactual to X_high_risk_tp
    #X_high_risk_tp = X_high_risk_tp.append(counterfactual, ignore_index=True)
    counterfactuals_df = counterfactuals_df.append(counterfactual, ignore_index=True)

In [ ]:
xgb_model.predict(counterfactuals_df)

In [ ]:
patient_cvd_counterfactual_ideal[3]

In [ ]:
validity_cfs,validity_list = get_validity(patient_cvd_counterfactual_ideal,xgb_pipeline)

In [ ]:
X_high_risk_tp[3:4]

In [ ]:
X_high_risk_tp[0:1]

In [ ]:
counterfactuals_df

In [ ]:
import json

selected_data = []

for patient in patient_cvd_counterfactual_ideal:
    patient_dict = json.loads(patient)
    measures = [item for sublist in patient_dict['Preventive Measures'] for item in sublist]
    cholestrol_values = [measure['Cholestrol'] for measure in measures if 'Cholestrol' in measure]
    blood_pressure_values = [measure['Resting Blood Pressure'] for measure in measures if 'Resting Blood Pressure' in measure]
    heart_rate_values = [measure['Max Heart Rate'] for measure in measures if 'Max Heart Rate' in measure]
    selected_data.append({
        'Cholestrol': cholestrol_values,
        'Resting Blood Pressure': blood_pressure_values,
        'Max Heart Rate': heart_rate_values
    })

In [ ]:
import numpy as np

statistics_llm = {}

for key in ['Cholestrol', 'Resting Blood Pressure', 'Max Heart Rate']:
    values = [int(value) for sublist in [patient[key] for patient in selected_data] for value in sublist]
    statistics_llm[key] = {
        'mean': np.mean(values),
        'median': np.median(values),
        'std_dev': np.std(values)
    }

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


# Convert the dictionary to a DataFrame
statistics_df_llm = pd.DataFrame(statistics_llm)

# Create a new figure
plt.figure(figsize=(15, 5))

# Plot the mean values
plt.subplot(1, 3, 1)
statistics_df_llm.loc['mean'].plot(kind='bar')
plt.title('Mean')
plt.ylabel('Value')

# Plot the median values
plt.subplot(1, 3, 2)
statistics_df_llm.loc['median'].plot(kind='bar')
plt.title('Median')

# Plot the standard deviation values
plt.subplot(1, 3, 3)
statistics_df_llm.loc['std_dev'].plot(kind='bar')
plt.title('Standard Deviation')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

# Convert the lists to pandas Series
cholesterol_series = pd.Series(cholesterol)
resting_bp_series = pd.Series(resting_bp)
max_heart_rate_series = pd.Series(max_heart_rate)

# Calculate the mean
cholesterol_mean = cholesterol_series.mean()
resting_bp_mean = resting_bp_series.mean()
max_heart_rate_mean = max_heart_rate_series.mean()

# Calculate the median
cholesterol_median = cholesterol_series.median()
resting_bp_median = resting_bp_series.median()
max_heart_rate_median = max_heart_rate_series.median()

# Calculate the standard deviation
cholesterol_std = cholesterol_series.std()
resting_bp_std = resting_bp_series.std()
max_heart_rate_std = max_heart_rate_series.std()

# Now the variables cholesterol_mean, resting_bp_mean, max_heart_rate_mean, cholesterol_median, resting_bp_median, max_heart_rate_median, cholesterol_std, resting_bp_std, and max_heart_rate_std contain the calculated values

For Single Counterfactuals

In [ ]:
# Print the mean values
print("Mean values:")
print("Cholesterol:", cholesterol_mean)
print("Resting BP:", resting_bp_mean)
print("Max Heart Rate:", max_heart_rate_mean)

# Print the median values
print("\nMedian values:")
print("Cholesterol:", cholesterol_median)
print("Resting BP:", resting_bp_median)
print("Max Heart Rate:", max_heart_rate_median)

# Print the standard deviation values
print("\nStandard deviation values:")
print("Cholesterol:", cholesterol_std)
print("Resting BP:", resting_bp_std)
print("Max Heart Rate:", max_heart_rate_std)

Feasible Counterfactuals -  For the instances that did not have Ideal CFs


In [ ]:
X_high_risk_tp = X_test[(y_pred == 1) & (y_test == 1)].reset_index().drop(['index'], axis=1)
#df_no_counterfactuals = X_high_risk_tp.loc[[exp[0] is None for exp in exps_ideal]]

In [ ]:
import threading
import queue

df_no_feasible_counterfactuals = pd.DataFrame(columns=df_no_counterfactuals.columns)

def generate_cf_feasible(test_instance, timeout=30):
    q = queue.Queue()

    def target():
        try:
            result = Dice(d, m, method='genetic').generate_counterfactuals(test_instance, total_CFs=5, desired_class="opposite",
                                                                           features_to_vary=["trestbps", "chol", "thalach"],
                                                                           #diversity_weight=5, proximity_weight=2, sparsity_weight=5,
                                                                           permitted_range={"trestbps": [80, 149],
                                                                                            "chol": [150, test_instance['chol'].values[0]-0.1*test_instance['chol'].values[0]],
                                                                                            "thalach": [120, 220 - test_instance['age'].values[0]]}                                                                           )
            q.put(result)
        except Exception as e:
            print("No counterfactuals found for test instance:", test_instance)
            df_no_feasible_counterfactuals.append(test_instance)
            q.put(None)

    # Start a new thread to run the target function
    thread = threading.Thread(target=target)
    thread.start()

    # Wait for the thread to finish or raise a timeout exception
    thread.join(timeout)

    if thread.is_alive():
        # The thread is still running, so raise a timeout exception
        print("No counterfactuals found for test instance-timed out:", test_instance)
        df_no_feasible_counterfactuals.append(test_instance)
        q.put(None)
    else:
        # The thread has finished, so return the result
        return q.get()

In [ ]:
exps_feasible = []
num_cores = -1
df_no_counterfactuals_feasible = pd.DataFrame(columns=X_high_risk_tp.columns)
# Iterate over each instance of X_high_risk and generate counterfactuals
for i in range(len(X_high_risk_tp)):
    test_instance = X_high_risk_tp.iloc[[i]]
    print(i)   
    exp = Parallel(n_jobs=num_cores)(delayed(generate_cf_feasible)(test_instance) for i in range(1))
    exps_feasible.append(exp)

In [ ]:
num_none = sum(exp[0] is None for exp in exps_feasible)
print("Number of None values in exps_feasible:", num_none)

In [ ]:
feasible_validity_list = []
for exps in exps_feasible:
    
    if exps[0] is not None:
       exp_df = exps[0].cf_examples_list[0].final_cfs_df
       feasible_validity_list.append(xgb_pipeline.predict(exp_df)[0])

In [ ]:
exps_feasible[0][0].visualize_as_dataframe()

In [ ]:
percentage = sum(feasible_validity_list) / len(feasible_validity_list) * 100
print("Number of Valid Feasible Counterfactuals: {:.2f}%".format(100-percentage))

In [ ]:
#df_no_counterfactuals_feasible = X_test_no_ideal_cf.loc[[exp[0] is None for exp in exps_feasible]]

In [ ]:
#df_no_counterfactuals_feasible

General CFs

In [ ]:
X_high_risk_tp = X_test[(y_pred == 1) & (y_test == 1)].reset_index().drop(['index'], axis=1)
#df_no_counterfactuals = X_high_risk_tp.loc[[exp[0] is None for exp in exps_ideal]]

In [ ]:
import threading
import queue



def generate_cf_general(test_instance, timeout=300):
    q = queue.Queue()

    def target():
        try:
            result = Dice(d, m, method='genetic').generate_counterfactuals(test_instance, total_CFs=20, desired_class="opposite",
                                                                           features_to_vary=["trestbps", "chol", "thalach"],
                                                                           diversity_weight=5, proximity_weight=2, sparsity_weight=5
                                                                          )
            q.put(result)
        except Exception as e:
            print("No counterfactuals found for test instance:", test_instance)
            #df_no_counterfactuals.append(test_instance)
            q.put(None)

    # Start a new thread to run the target function
    thread = threading.Thread(target=target)
    thread.start()

    # Wait for the thread to finish or raise a timeout exception
    thread.join(timeout)

    if thread.is_alive():
        # The thread is still running, so raise a timeout exception
        print("No counterfactuals found for test instance-timed out:", test_instance)
        #df_no_counterfactuals.append(test_instance)
        q.put(None)
    else:
        # The thread has finished, so return the result
        return q.get()

In [ ]:
exps_general= []
num_cores = -1
df_no_counterfactuals_general= pd.DataFrame(columns=X_high_risk_tp.columns)
# Iterate over each instance of X_high_risk and generate counterfactuals
for i in range(len(X_high_risk_tp)):
    test_instance = X_high_risk_tp.iloc[[i]]
    print(i)   
    exp = Parallel(n_jobs=num_cores)(delayed(generate_cf_general)(test_instance) for i in range(1))
    exps_general.append(exp)

In [ ]:
num_none = sum(exp[0] is None for exp in exps_general)
print("Number of None values in exps_general:", num_none)

In [ ]:
exps_general[0][0].visualize_as_dataframe()

In [ ]:
general_validity_list = []
for exps in exps_general:
    
    if exps[0] is not None:
       exp_df = exps[0].cf_examples_list[0].final_cfs_df
       general_validity_list.append(xgb_pipeline.predict(exp_df)[0])

In [ ]:
percentage = sum(general_validity_list) / len(general_validity_list) * 100
print("Number of Valid General Counterfactuals: {:.2f}%".format(100-percentage))

In [ ]:
for i in range(len(X_high_risk_tp)):
     
     test_instance = X_high_risk_tp.iloc[[i]]
     exp = exps_general[i] 

     if exp[0] is not None:
          exp_dfs = exp[0].cf_examples_list[0].final_cfs_df
          thalach_test = test_instance['thalach'].values[0] 
          chol_test = test_instance['chol'].values[0]


          feasible_counterfactuals = exp_dfs[((exp_dfs['thalach'] > thalach_test) & (exp_dfs['thalach']< 220 - exp_dfs['age'])) & (exp_dfs['chol'] <= chol_test-0.1*chol_test) & (exp_dfs['trestbps'] <= 149)]

          if len(feasible_counterfactuals) > 0:
               print(i)

In [ ]:
feasible_counterfactuals

In [ ]:
exp_dfs = exps_general[1][0].cf_examples_list[0].final_cfs_df


feasible_counterfactuals = exp_dfs[((exp_dfs['thalach'] > thalach_test) & (exp_dfs['thalach']< 220 - exp_dfs['age'])) & (exp_dfs['chol'] <= 200) & (exp_dfs['trestbps'] <= 120)]

In [ ]:
feasible_counterfactuals

In [ ]:
exps_general[1][0].visualize_as_dataframe()

 Less Feasible Counterfactuals

In [ ]:
X_high_risk_tp = X_test[(y_pred == 1) & (y_test == 1)].reset_index().drop(['index'], axis=1)
#df_no_counterfactuals = X_high_risk_tp.loc[[exp[0] is None for exp in exps_ideal]]

In [ ]:
import threading
import queue



def generate_cf_general_feasible(test_instance, timeout=300):
    q = queue.Queue()

    def target():
        try:
            result = Dice(d, m, method='genetic').generate_counterfactuals(test_instance, total_CFs=20, desired_class="opposite",
                                                                           features_to_vary=["trestbps", "chol", "thalach"],
                                                                           diversity_weight=5, proximity_weight=2, sparsity_weight=5,
                                                                           permitted_range={"trestbps": [80, test_instance['age'].values[0]-1],
                                                                                            "chol": [150, test_instance['chol'].values[0]-1],
                                                                                            "thalach": [test_instance['thalach'].values[0]+1, 220 - test_instance['age'].values[0]]}
                                                                          )
            q.put(result)
        except Exception as e:
            print("No counterfactuals found for test instance:", test_instance)
            #df_no_counterfactuals.append(test_instance)
            q.put(None)

    # Start a new thread to run the target function
    thread = threading.Thread(target=target)
    thread.start()

    # Wait for the thread to finish or raise a timeout exception
    thread.join(timeout)

    if thread.is_alive():
        # The thread is still running, so raise a timeout exception
        print("No counterfactuals found for test instance-timed out:", test_instance)
        #df_no_counterfactuals.append(test_instance)
        q.put(None)
    else:
        # The thread has finished, so return the result
        return q.get()

In [ ]:
exps_general_feasible= []
num_cores = -1
df_no_counterfactuals_general_feasible= pd.DataFrame(columns=X_high_risk_tp.columns)
# Iterate over each instance of X_high_risk and generate counterfactuals
for i in range(len(X_high_risk_tp)):
    test_instance = X_high_risk_tp.iloc[[i]]
    print(i)   
    exp = Parallel(n_jobs=num_cores)(delayed(generate_cf_general_feasible)(test_instance) for i in range(1))
    exps_general_feasible.append(exp)

In [ ]:
num_none = sum(exp[0] is None for exp in exps_general_feasible)
print("Number of None values in exps_general_feasible:", num_none)

In [ ]:
general_feasible_validity_list = []
for exps in exps_general_feasible:
    
    if exps[0] is not None:
       exp_df = exps[0].cf_examples_list[0].final_cfs_df
       general_feasible_validity_list.append(xgb_pipeline.predict(exp_df)[0])

In [ ]:
percentage = sum(general_feasible_validity_list) / len(general_feasible_validity_list) * 100
print("Number of Valid General Counterfactuals: {:.2f}%".format(100-percentage))